In [1]:
import os
import pathlib
import pprint
import json
import itertools
import shutil

import math

In [2]:
def save_json_dict(
        json_filename,
        json_dict
        ):

    with open(json_filename, 'w') as json_file:
        json.dump(json_dict, json_file, indent=4)


def load_json_dict(
        json_filename
        ):

    with open(json_filename, 'r') as json_file:
        json_dict = json.load(json_file)

    return json_dict

In [3]:
train_dataset_len = 98832
val_dataset_len = 48935

---

# Create copies

In [4]:
data_dir = os.path.join(pathlib.Path.home(), "data", "fashion_retrieval", "ret_train", "convnext_t")

for data_subdir in os.listdir(data_dir):

    data_filename = os.path.join(data_dir, data_subdir, "exp_data.json")
    corr_data_filename = os.path.join(data_dir, data_subdir, "exp_data_corr.json")
    
    shutil.copyfile(data_filename, corr_data_filename)

#

data_dir = os.path.join(pathlib.Path.home(), "data", "fashion_retrieval", "ret_train", "resnet_50")

for data_subdir in os.listdir(data_dir):

    data_filename = os.path.join(data_dir, data_subdir, "exp_data.json")
    corr_data_filename = os.path.join(data_dir, data_subdir, "exp_data_corr.json")
    
    shutil.copyfile(data_filename, corr_data_filename)

---

# ResNet50

In [5]:
data_dir = os.path.join(pathlib.Path.home(), "data", "fashion_retrieval", "ret_train", "resnet_50")

for data_subdir in os.listdir(data_dir):

    data_filename = os.path.join(data_dir, data_subdir, "exp_data_corr.json")
    print(data_filename)

    data = load_json_dict(data_filename)

    #

    num_gpus = len(data["settings"]["gpu_usage"])

    batch_size_1 = data["settings"]["stage_1"]["data_loading"]["batch_size"]
    grad_acc_iters_1 = data["settings"]["stage_1"].get("max_acc_iter", 1)
    corr_lr_1 = data["settings"]["stage_1"]["learning_rate_list"][0]

    batch_size_2 = data["settings"]["stage_2"]["data_loading"]["batch_size"]
    grad_acc_iters_2 = data["settings"]["stage_2"].get("max_acc_iter", 1)
    corr_lr_2 = data["settings"]["stage_2"]["learning_rate_list"][0]

    actual_batch_size_1 = batch_size_1 * grad_acc_iters_1 * num_gpus
    actual_batch_size_2 = batch_size_2 * grad_acc_iters_2 * num_gpus

    base_lr_1 = corr_lr_1 / actual_batch_size_1
    base_lr_2 = corr_lr_2 / actual_batch_size_2

    #

    if "max_acc_iter" in data["settings"]["stage_1"].keys():
        del data["settings"]["stage_1"]["max_acc_iter"]
        data["settings"]["stage_1"]["data_loading"]["grad_acc_iters"] = grad_acc_iters_1

    if "max_acc_iter" in data["settings"]["stage_2"].keys():
        del data["settings"]["stage_2"]["max_acc_iter"]
        data["settings"]["stage_2"]["data_loading"]["grad_acc_iters"] = grad_acc_iters_2

    data["settings"]["stage_1"]["base_lr"] = base_lr_1
    data["settings"]["stage_2"]["base_lr"] = base_lr_2

    for idx in range(len(data["results"]["stage_1"]["train_mean_loss_list"])):
        for iidx in range(len(data["results"]["stage_1"]["train_mean_loss_list"][idx])):
            data["results"]["stage_1"]["train_mean_loss_list"][idx][iidx] *= math.ceil(train_dataset_len / actual_batch_size_1)
            data["results"]["stage_1"]["train_mean_loss_list"][idx][iidx] /= train_dataset_len

    for idx in range(len(data["results"]["stage_1"]["val_mean_loss_list"])):
        for iidx in range(len(data["results"]["stage_1"]["val_mean_loss_list"][idx])):
            data["results"]["stage_1"]["val_mean_loss_list"][idx][iidx] *= math.ceil(val_dataset_len / actual_batch_size_1)
            data["results"]["stage_1"]["val_mean_loss_list"][idx][iidx] /= val_dataset_len

    for idx in range(len(data["results"]["stage_2"]["train_mean_loss_list"])):
        for iidx in range(len(data["results"]["stage_2"]["train_mean_loss_list"][idx])):
            data["results"]["stage_2"]["train_mean_loss_list"][idx][iidx] *= math.ceil(train_dataset_len / actual_batch_size_2)
            data["results"]["stage_2"]["train_mean_loss_list"][idx][iidx] /= train_dataset_len

    for idx in range(len(data["results"]["stage_2"]["val_mean_loss_list"])):
        for iidx in range(len(data["results"]["stage_2"]["val_mean_loss_list"][idx])):
            data["results"]["stage_2"]["val_mean_loss_list"][idx][iidx] *= math.ceil(val_dataset_len / actual_batch_size_2)
            data["results"]["stage_2"]["val_mean_loss_list"][idx][iidx] /= val_dataset_len

    save_json_dict(data_filename, data)

/home-net/gortega/data/fashion_retrieval/ret_train/resnet_50/test_006/exp_data_corr.json
/home-net/gortega/data/fashion_retrieval/ret_train/resnet_50/test_003/exp_data_corr.json
/home-net/gortega/data/fashion_retrieval/ret_train/resnet_50/test_002/exp_data_corr.json
/home-net/gortega/data/fashion_retrieval/ret_train/resnet_50/test_005/exp_data_corr.json
/home-net/gortega/data/fashion_retrieval/ret_train/resnet_50/test_007/exp_data_corr.json
/home-net/gortega/data/fashion_retrieval/ret_train/resnet_50/test_004/exp_data_corr.json
/home-net/gortega/data/fashion_retrieval/ret_train/resnet_50/test_000/exp_data_corr.json
/home-net/gortega/data/fashion_retrieval/ret_train/resnet_50/test_001/exp_data_corr.json
/home-net/gortega/data/fashion_retrieval/ret_train/resnet_50/test_008/exp_data_corr.json


# ConvNeXt Tiny

In [6]:
data_dir = os.path.join(pathlib.Path.home(), "data", "fashion_retrieval", "ret_train", "convnext_t")

for data_subdir in os.listdir(data_dir):

    data_filename = os.path.join(data_dir, data_subdir, "exp_data_corr.json")
    print(data_filename)

    data = load_json_dict(data_filename)

    #

    num_gpus = len(data["settings"]["gpu_usage"])

    batch_size_1 = data["settings"]["stage_1"]["data_loading"]["batch_size"]
    grad_acc_iters_1 = data["settings"]["stage_1"].get("max_acc_iter", 1)
    corr_lr_1 = data["settings"]["stage_1"]["learning_rate_list"][0]

    batch_size_2 = data["settings"]["stage_2"]["data_loading"]["batch_size"]
    grad_acc_iters_2 = data["settings"]["stage_2"].get("max_acc_iter", 1)
    corr_lr_2 = data["settings"]["stage_2"]["learning_rate_list"][0]

    actual_batch_size_1 = batch_size_1 * grad_acc_iters_1 * num_gpus
    actual_batch_size_2 = batch_size_2 * grad_acc_iters_2 * num_gpus

    base_lr_1 = corr_lr_1 / actual_batch_size_1
    base_lr_2 = corr_lr_2 / actual_batch_size_2

    if "012" in data_filename or "000" in data_filename:
        print("batch_size_1", batch_size_1)
        print("grad_acc_iters_1", grad_acc_iters_1)
        print("num_gpus", num_gpus)
        print("actual_batch_size_1", actual_batch_size_1)
        print("corr_lr_1", corr_lr_1)
        print("base_lr_1", base_lr_1)

    #

    if "max_acc_iter" in data["settings"]["stage_1"].keys():
        del data["settings"]["stage_1"]["max_acc_iter"]
        data["settings"]["stage_1"]["data_loading"]["grad_acc_iters"] = grad_acc_iters_1

    if "max_acc_iter" in data["settings"]["stage_2"].keys():
        del data["settings"]["stage_2"]["max_acc_iter"]
        data["settings"]["stage_2"]["data_loading"]["grad_acc_iters"] = grad_acc_iters_2

    data["settings"]["stage_1"]["base_lr"] = base_lr_1
    data["settings"]["stage_2"]["base_lr"] = base_lr_2

    for idx in range(len(data["results"]["stage_1"]["train_mean_loss_list"])):
        for iidx in range(len(data["results"]["stage_1"]["train_mean_loss_list"][idx])):
            data["results"]["stage_1"]["train_mean_loss_list"][idx][iidx] *= math.ceil(train_dataset_len / actual_batch_size_1)
            data["results"]["stage_1"]["train_mean_loss_list"][idx][iidx] /= train_dataset_len

    for idx in range(len(data["results"]["stage_1"]["val_mean_loss_list"])):
        for iidx in range(len(data["results"]["stage_1"]["val_mean_loss_list"][idx])):
            data["results"]["stage_1"]["val_mean_loss_list"][idx][iidx] *= math.ceil(val_dataset_len / actual_batch_size_1)
            data["results"]["stage_1"]["val_mean_loss_list"][idx][iidx] /= val_dataset_len

    for idx in range(len(data["results"]["stage_2"]["train_mean_loss_list"])):
        for iidx in range(len(data["results"]["stage_2"]["train_mean_loss_list"][idx])):
            data["results"]["stage_2"]["train_mean_loss_list"][idx][iidx] *= math.ceil(train_dataset_len / actual_batch_size_2)
            data["results"]["stage_2"]["train_mean_loss_list"][idx][iidx] /= train_dataset_len

    for idx in range(len(data["results"]["stage_2"]["val_mean_loss_list"])):
        for iidx in range(len(data["results"]["stage_2"]["val_mean_loss_list"][idx])):
            data["results"]["stage_2"]["val_mean_loss_list"][idx][iidx] *= math.ceil(val_dataset_len / actual_batch_size_2)
            data["results"]["stage_2"]["val_mean_loss_list"][idx][iidx] /= val_dataset_len

    save_json_dict(data_filename, data)

/home-net/gortega/data/fashion_retrieval/ret_train/convnext_t/test_006/exp_data_corr.json
/home-net/gortega/data/fashion_retrieval/ret_train/convnext_t/test_011/exp_data_corr.json
/home-net/gortega/data/fashion_retrieval/ret_train/convnext_t/test_010/exp_data_corr.json
/home-net/gortega/data/fashion_retrieval/ret_train/convnext_t/test_003/exp_data_corr.json
/home-net/gortega/data/fashion_retrieval/ret_train/convnext_t/test_002/exp_data_corr.json
/home-net/gortega/data/fashion_retrieval/ret_train/convnext_t/test_005/exp_data_corr.json
/home-net/gortega/data/fashion_retrieval/ret_train/convnext_t/test_012/exp_data_corr.json
batch_size_1 16
grad_acc_iters_1 1
num_gpus 8
actual_batch_size_1 128
corr_lr_1 0.0076
base_lr_1 5.9375e-05
/home-net/gortega/data/fashion_retrieval/ret_train/convnext_t/test_007/exp_data_corr.json
/home-net/gortega/data/fashion_retrieval/ret_train/convnext_t/test_004/exp_data_corr.json
/home-net/gortega/data/fashion_retrieval/ret_train/convnext_t/test_000/exp_data_co